In [1]:
import pandas as pd
import numpy as np
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import math 
import seaborn as sns
import matplotlib.colors as mcolors
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from statsmodels.formula.api import mixedlm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import os
import matplotlib.pyplot as mpl
import matplotlib

colors = list(mcolors.TABLEAU_COLORS.keys())*2

parentDirectory = os.path.abspath(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir))
DATA_DIR = parentDirectory +'/data/'
FIGURES_DIR = parentDirectory +'/figures/'

full_names = {
    'AU': 'Australia',
    'BR': 'Brazil',
    'CA': 'Canada',
    'FR': 'France',
    'DE': 'Germany',
    'IN': 'India',
    'IT': 'Italy',
    'MX': 'Mexico',
    'ES': 'Spain',
    'GB': 'United Kingdom',
    'US': 'United States',
    'DK': 'Denmark'
}

event_dicts = [{'country': 'AU',
  'start_md_1': '2020-03-27',
  'end_md_1': '2020-06-07',
  'start_md_2': np.nan},
 {'country': 'BR',
  'start_md_1': '2020-03-23',
  'end_md_1': '2020-08-09',
  'start_md_2': np.nan},
 {'country': 'CA',
  'start_md_1': '2020-03-19',
  'end_md_1': '2020-06-21',
  'start_md_2': '2020-10-12'},
 {'country': 'DE',
  'start_md_1': '2020-03-21',
  'end_md_1': '2020-05-09',
  'start_md_2': '2020-12-18'},
 {'country': 'DK',
  'start_md_1': '2020-03-17',
  'end_md_1': '2020-05-07',
  'start_md_2': np.nan},
 {'country': 'ES',
  'start_md_1': '2020-03-17',
  'end_md_1': '2020-06-14',
  'start_md_2': '2020-11-07'},
 {'country': 'FR',
  'start_md_1': '2020-03-18',
  'end_md_1': '2020-06-08',
  'start_md_2': '2020-11-01'},
 {'country': 'GB',
  'start_md_1': '2020-03-23',
  'end_md_1': '2020-08-03',
  'start_md_2': '2020-10-21'},
 {'country': 'IN',
  'start_md_1': '2020-03-24',
  'end_md_1': '2020-10-29',
  'start_md_2': np.nan},
 {'country': 'IT',
  'start_md_1': '2020-03-11',
  'end_md_1': '2020-06-06',
  'start_md_2': '2020-11-06'},
 {'country': 'JP',
  'start_md_1': '2020-04-12',
  'end_md_1': '2020-05-30',
  'start_md_2': np.nan},
 {'country': 'KE',
  'start_md_1': '2020-03-24',
  'end_md_1': '2020-10-04',
  'start_md_2': np.nan},
 {'country': 'MX',
  'start_md_1': '2020-03-25',
  'end_md_1': '2020-10-06',
  'start_md_2': np.nan},
 {'country': 'NG',
  'start_md_1': '2020-03-27',
  'end_md_1': '2020-08-09',
  'start_md_2': np.nan},
 {'country': 'US',
  'start_md_1': '2020-03-21',
  'end_md_1': '2020-06-11',
  'start_md_2': '2020-11-26'}]

df_events = pd.DataFrame(event_dicts)

df_events['start_md_1'] = pd.to_datetime(df_events['start_md_1'])
df_events['end_md_1'] = pd.to_datetime(df_events['end_md_1'])
df_events['start_md_2'] = pd.to_datetime(df_events['start_md_2'])

df_agg = pd.read_pickle(DATA_DIR+'df_agg_cats.pickle')

In [2]:
#helpers

def generate_equation(order):
    if order == 'Cubic':
        eq = "volume_total ~ intervention_flag*k*year + intervention_flag*np.power(k,2)*year + intervention_flag*np.power(k,3)*year"
    elif order == "Quadratic":
        eq = "volume_total ~ intervention_flag*k*year + intervention_flag*np.power(k,2)*year"
    elif order == "Linear":
        eq = "volume_total ~ intervention_flag*k*year"
    elif order == 'Constant':
        eq = "volume_total ~ intervention_flag*year"
    return eq

def generate_equation_interactions(order):
    if order == 'Cubic':
        eq = "volume_total ~ intervention_flag*k*year*C(country)*C(category) + intervention_flag*np.power(k,2)*year*C(country)*C(category) + intervention_flag*np.power(k,3)*year*C(country)*C(category)"
    elif order == "Quadratic":
        eq = "volume_total ~ intervention_flag*k*year*C(country)*C(category) + intervention_flag*np.power(k,2)*year*C(country)*C(category)"
    elif order == "Linear":
        eq = "volume_total ~ intervention_flag*k*year*C(country)*C(category)"
    elif order == 'Constant':
        eq = "volume_total ~ intervention_flag*year*C(country)*C(category)"
    return eq
    
def get_standard_error_sum(covariates):
    '''
    #95CI is approximated with +- 2 sum_variance_standard_error
    '''
    
    #get the variance covariance matrix
    vcov = result_interactions.cov_params()\
        .loc[covariates,covariates].values
    
    #calculate the sum of all pair wise covariances by summing up
    m_sum = np.sum(vcov)
    
    #variance of a sum of variables is the square root
    return np.sqrt((m_sum))

def make_stars(val):
    if val<0.0001:
        return '****'
    elif val<0.001:
        return '***'
    elif val<0.01:
        return '**'
    elif val<0.05:
        return '*'
    else:
        return ''

def make_star_ste(value,ste):
    if value>0 and value-2*ste>0:
        return '*'
    elif value<0 and value+2*ste<0:
        return '*'
    else:
        return ''

In [3]:
weeks_2019 = list(df_agg.iloc[0]['volume_weekly_total'].index)[:52]
weeks_2020 = list(df_agg.iloc[0]['volume_weekly_total'].index)[52:]

In [4]:
l = []
for cnt, row in df_agg.iterrows():
    start_md = df_events.loc[df_events['country'] == row['country']].iloc[0]['start_md_1']
    end_md = df_events.loc[df_events['country'] == row['country']].iloc[0]['end_md_1']
    start_md2 = df_events.loc[df_events['country'] == row['country']].iloc[0]['start_md_2']
    
    for week in zip(row['volume_weekly_total'].index,row['volume_weekly_total'].values,row['volume_percent_weekly_total'].values):
        
        entry = {}

        entry['country'] = row['country']
        entry['category'] = row['category']
        

        if week[0] in weeks_2020:
            date = pd.to_datetime(week[0])

            if type(start_md2)!=pd._libs.tslibs.nattype.NaTType and date > start_md2:
                continue

            entry['k'] = math.floor(((date - start_md).days +7) / 7)
            entry['volume_total'] = week[1]
            entry['volume_percent'] = week[2]
            entry['year'] = '2020'
            l.append(entry)

        elif week[0] in weeks_2019:
            date = pd.to_datetime(weeks_2020[weeks_2019.index(week[0])])
            
            if type(start_md2)!=pd._libs.tslibs.nattype.NaTType and date > start_md2:
                continue

            entry['k'] = math.floor(((date - start_md).days +7) / 7)
            entry['volume_total'] = week[1]
            entry['volume_percent'] = week[2]
            entry['year'] = '2019'
            l.append(entry)

df = pd.DataFrame(l)      

In [5]:
df = df.loc[(df['k'] >= -30) & (df['k'] <= 30)]
df = df.loc[(df['country'].isin(list(full_names.keys())))]
df['intervention_flag'] = df['k'].apply(lambda x: 1 if x >= 0 else 0)

In [6]:
cats = list(df['category'].unique())
k = 30

In [7]:
df_temp = df.loc[(df['k'] >= -k) & (df['k'] <= k)].copy()
df_temp['volume_total'] = df_temp['volume_total'].apply(np.log)

In [28]:
entries_list = []

for name, group in df_temp.groupby(['country','category']):
    entry = {}
    
    mod = smf.ols(generate_equation('Quadratic'), data = group)
    res = mod.fit(cov_type='hc0')
    
    entry['country'] = name[0]
    entry['category'] = name[1]
    
    entry['alpha'] = res.params['intervention_flag:year[T.2020]']
    entry['ste'] = res.bse['intervention_flag:year[T.2020]']
    entry['pval'] = res.pvalues['intervention_flag:year[T.2020]']
    entry['r2'] = res.rsquared
    
    entries_list.append(entry)

In [29]:
df_res = pd.DataFrame(entries_list)

In [33]:
df_res.loc[df_res['category']=='pastry and bakery product']

,country,category,alpha,ste,pval,r2
12,AU,pastry and bakery product,0.519702,0.173019,2.666950e-03,0.856903
40,BR,pastry and bakery product,0.909112,0.086046,4.308115e-26,0.928356
68,CA,pastry and bakery product,1.256330,0.246688,3.528296e-07,0.808006
96,DE,pastry and bakery product,0.960700,0.154088,4.525399e-10,0.871245
124,DK,pastry and bakery product,0.428403,0.196029,2.885878e-02,0.553069
152,ES,pastry and bakery product,1.446911,0.246645,4.454636e-09,0.862044
180,FR,pastry and bakery product,1.315179,0.273901,1.573627e-06,0.816687
208,GB,pastry and bakery product,1.097855,0.225667,1.144944e-06,0.927841
236,IN,pastry and bakery product,1.053318,0.243026,1.463055e-05,0.744596
264,IT,pastry and bakery product,1.084825,0.332984,1.122453e-03,0.842861


In [73]:
for name, group in df_res.groupby(['category']):
    print(name+' &')
    c=0
    for name1, group1 in group.groupby(['country']):
        c+=1
        if c ==12:
            print(str(round(group1['alpha'].values[0],2))+ '['+str(round(group1['ste'].values[0],2))+
              '], '+str(round(group1['r2'].values[0],2)))
        else:
            print(str(round(group1['alpha'].values[0],2))+ '['+str(round(group1['ste'].values[0],2))+
              '], '+str(round(group1['r2'].values[0],2))+' &')
    print('\\\\')

beef dish &
0.16[0.08], 0.79 &
0.35[0.06], 0.82 &
0.24[0.08], 0.63 &
0.26[0.1], 0.4 &
0.16[0.15], 0.39 &
0.59[0.13], 0.7 &
0.38[0.15], 0.68 &
0.55[0.12], 0.77 &
0.23[0.11], 0.64 &
0.17[0.13], 0.39 &
0.2[0.1], 0.66 &
0.41[0.11], 0.66
\\
bread and flatbread &
0.39[0.15], 0.91 &
0.6[0.11], 0.96 &
1.08[0.17], 0.9 &
0.48[0.11], 0.93 &
0.69[0.17], 0.88 &
1.41[0.21], 0.91 &
1.36[0.19], 0.9 &
0.67[0.25], 0.96 &
0.64[0.13], 0.8 &
1.37[0.28], 0.86 &
0.13[0.11], 0.92 &
0.84[0.09], 0.92
\\
cheese &
0.37[0.11], 0.74 &
0.45[0.09], 0.85 &
0.35[0.11], 0.68 &
0.18[0.1], 0.8 &
0.26[0.21], 0.52 &
0.64[0.18], 0.65 &
0.44[0.15], 0.6 &
0.62[0.12], 0.76 &
0.6[0.11], 0.89 &
0.41[0.18], 0.64 &
0.62[0.12], 0.66 &
0.27[0.08], 0.66
\\
chicken dish &
0.25[0.09], 0.84 &
0.28[0.06], 0.91 &
0.3[0.09], 0.81 &
0.31[0.12], 0.77 &
0.19[0.13], 0.65 &
0.77[0.14], 0.87 &
0.52[0.16], 0.67 &
0.57[0.11], 0.82 &
0.38[0.13], 0.79 &
0.5[0.17], 0.81 &
0.34[0.07], 0.91 &
0.29[0.07], 0.83
\\
cocktail &
0.25[0.16], 0.73 &
0.35[0.21],

In [85]:
df_agg = pd.read_pickle(DATA_DIR+'modes_coarse.pickle')

In [86]:
l = []
for cnt, row in df_agg.iterrows():
    start_md = df_events.loc[df_events['country'] == row['country']].iloc[0]['start_md_1']
    end_md = df_events.loc[df_events['country'] == row['country']].iloc[0]['end_md_1']
    start_md2 = df_events.loc[df_events['country'] == row['country']].iloc[0]['start_md_2']
    
    for week in zip(row['volume_weekly_total'].index,row['volume_weekly_total'].values,row['volume_percent_weekly_total'].values):
        
        entry = {}

        entry['country'] = row['country']
        entry['category'] = row['category']
        

        if week[0] in weeks_2020:
            date = pd.to_datetime(week[0])

            if type(start_md2)!=pd._libs.tslibs.nattype.NaTType and date > start_md2:
                continue

            entry['k'] = math.floor(((date - start_md).days +7) / 7)
            entry['volume_total'] = week[1]
            entry['volume_percent'] = week[2]
            entry['year'] = '2020'
            l.append(entry)

        elif week[0] in weeks_2019:
            date = pd.to_datetime(weeks_2020[weeks_2019.index(week[0])])
            
            if type(start_md2)!=pd._libs.tslibs.nattype.NaTType and date > start_md2:
                continue

            entry['k'] = math.floor(((date - start_md).days +7) / 7)
            entry['volume_total'] = week[1]
            entry['volume_percent'] = week[2]
            entry['year'] = '2019'
            l.append(entry)

In [87]:
df = pd.DataFrame(l)
df = df.loc[(df['k'] >= -30) & (df['k'] <= 30)]
df = df.loc[(df['country'].isin(list(full_names.keys())))]
df['intervention_flag'] = df['k'].apply(lambda x: 1 if x >= 0 else 0)

In [88]:
cats = list(df['category'].unique())
k = 30

In [89]:
df_temp = df.loc[(df['k'] >= -k) & (df['k'] <= k)].copy()
df_temp['volume_total'] = df_temp['volume_total'].apply(np.log)

In [90]:
entries_list = []

for name, group in df_temp.groupby(['country','category']):
    entry = {}
    
    mod = smf.ols(generate_equation('Quadratic'), data = group)
    res = mod.fit(cov_type='hc0')
    
    entry['country'] = name[0]
    entry['category'] = name[1]
    
    entry['alpha'] = res.params['intervention_flag:year[T.2020]']
    entry['ste'] = res.bse['intervention_flag:year[T.2020]']
    entry['pval'] = res.pvalues['intervention_flag:year[T.2020]']
    entry['r2'] = res.rsquared
    
    entries_list.append(entry)

In [91]:
df_res = pd.DataFrame(entries_list)

In [93]:
for name, group in df_res.groupby(['category']):
    print(name+' &')
    c=0
    for name1, group1 in group.groupby(['country']):
        c+=1
        if c ==12:
            print(str(round(group1['alpha'].values[0],2))+ '['+str(round(group1['ste'].values[0],2))+
              '], '+str(round(group1['r2'].values[0],2)))
        else:
            print(str(round(group1['alpha'].values[0],2))+ '['+str(round(group1['ste'].values[0],2))+
              '], '+str(round(group1['r2'].values[0],2))+' &')
    print('\\\\')

Mode 1 &
0.34[0.11], 0.9 &
0.46[0.13], 0.9 &
0.57[0.1], 0.87 &
0.43[0.12], 0.77 &
0.29[0.08], 0.77 &
0.84[0.13], 0.9 &
0.73[0.15], 0.81 &
0.78[0.14], 0.94 &
1.03[0.12], 0.95 &
0.77[0.16], 0.87 &
0.58[0.11], 0.93 &
0.6[0.08], 0.91
\\
Mode 2 &
0.78[0.12], 0.9 &
0.7[0.92], 0.93 &
0.97[0.13], 0.95 &
0.15[0.38], 0.85 &
1.05[0.32], 0.9 &
-0.09[0.23], 0.66 &
0.96[0.35], 0.57 &
0.37[0.26], 0.86 &
0.37[0.15], 0.24 &
0.95[0.16], 0.7 &
0.4[0.3], 0.65 &
1.11[0.08], 0.96
\\
Mode 3 &
-0.78[0.15], 0.86 &
-0.23[0.17], 0.88 &
-0.68[0.17], 0.87 &
-0.98[0.21], 0.84 &
-0.7[0.19], 0.75 &
-1.65[0.23], 0.93 &
-1.7[0.19], 0.92 &
-0.95[0.18], 0.84 &
-0.49[0.15], 0.97 &
-1.45[0.26], 0.88 &
-0.54[0.15], 0.93 &
-0.39[0.16], 0.82
\\
Mode 4 &
0.1[0.17], 0.7 &
0.16[0.13], 0.85 &
0.24[0.11], 0.97 &
0.44[0.21], 0.7 &
0.19[0.24], 0.55 &
-0.25[0.21], 0.75 &
-0.58[0.37], 0.77 &
0.92[0.29], 0.87 &
-0.36[0.18], 0.92 &
-0.6[0.21], 0.72 &
-0.11[0.25], 0.49 &
-0.06[0.08], 0.84
\\
